In [ ]:
import pandas as pd
df = pd.read_excel('./data/data.xlsx')

df['date'] = pd.to_datetime(df['date'])

# Basic info
print(df.info())
print(df.describe())

In [ ]:
print(df.isna().sum())

## Clean NaN values

In [ ]:
# Drop rows with missing values
df = df.dropna()

print(df.isna().sum())
print("Length of data after dropping missing values: ", len(df))

In [ ]:
print(df.describe())

In [ ]:
import matplotlib.pyplot as plt

# plot precipitation + irrigation and potential evapotranspiration (mm) over time
plt.figure(figsize=(10, 4))
plt.plot(df['date'], df['precipitation + irrigation (mm)'], label='Precip + Irrigation')
plt.plot(df['date'], df['potential evapotranspiration (mm)'], label='Potential Evapotranspiration')
plt.title('Daily Water Input vs Potential Evapotranspiration')
plt.xlabel('Date')
plt.ylabel('mm')
plt.legend()
plt.show()

In [ ]:
# Plot soil moisture at various depths
plt.figure(figsize=(10, 4))
for depth_col in ['depth 10cm', 'depth 30cm', 'depth 60cm', 'depth 90cm']:
    plt.plot(df['date'], df[depth_col], label=depth_col)
plt.title('Soil Moisture at Different Depths')
plt.xlabel('Date')
plt.ylabel('Moisture Level')
plt.legend()
plt.show()


In [ ]:
# actual evapotranspiration (mm),groundwater recharge (mm)
plt.figure(figsize=(10, 4))
plt.plot(df['date'], df['actual evapotranspiration (mm)'], label='Actual Evapotranspiration')
plt.plot(df['date'], df['groundwater recharge (mm)'], label='Groundwater Recharge')
plt.title('Actual Evapotranspiration vs Groundwater Recharge')
plt.xlabel('Date')
plt.ylabel('mm')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

# Select numeric columns for correlation
cols = [
    'precipitation + irrigation (mm)',
    'potential evapotranspiration (mm)',
    'depth 10cm',
    'depth 30cm',
    'depth 60cm',
    'depth 90cm',
    'actual evapotranspiration (mm)',
    'groundwater recharge (mm)'
]
corr = df[cols].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Save the cleaned data
df.to_csv('./data/cleaned_data.csv', index=False)

# Seasonal Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
df = df.sort_values(by='date').reset_index(drop=True)

df_seasonal = df.rename(columns={
    'precipitation + irrigation (mm)': 'precip',
    'potential evapotranspiration (mm)': 'pet',
    'depth 10cm': 'moisture_10cm',
    'depth 30cm': 'moisture_30cm',
    'depth 60cm': 'moisture_60cm',
    'depth 90cm': 'moisture_90cm',
    'actual evapotranspiration (mm)': 'aet',
    'groundwater recharge (mm)': 'gw_recharge'
})

print(df_seasonal.head())


In [ ]:
def get_season(row_date):
    """
    Assigns a season based on the month (meteorological seasons).
    - Winter: Dec (12), Jan (1), Feb (2)
    - Spring: Mar (3), Apr (4), May (5)
    - Summer: Jun (6), Jul (7), Aug (8)
    - Autumn: Sep (9), Oct (10), Nov (11)
    """
    month = row_date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Create a new column for Season
df_seasonal['season'] = df_seasonal['date'].apply(get_season)

# Group by Season and get mean or other stats
seasonal_stats = df_seasonal.groupby('season').agg({
    'precip': 'mean',
    'pet': 'mean',
    'moisture_10cm': 'mean',
    'moisture_30cm': 'mean',
    'moisture_60cm': 'mean',
    'moisture_90cm': 'mean',
    'aet': 'mean',
    'gw_recharge': 'mean'
}).reset_index()

# sort by season order
season_order = ['Winter', 'Spring', 'Summer', 'Autumn']
seasonal_stats['season'] = pd.Categorical(seasonal_stats['season'], categories=season_order, ordered=True)
seasonal_stats = seasonal_stats.sort_values('season')

print("Seasonal Mean Values:")
print(seasonal_stats)

In [ ]:
# visualize the seasonal stats
plt.figure(figsize=(12, 6))
for col in ['precip', 'pet', 'moisture_10cm', 'moisture_30cm', 'moisture_60cm', 'moisture_90cm', 'aet', 'gw_recharge']:
    sns.lineplot(x='season', y=col, data=seasonal_stats, marker='o', label=col)
plt.title('Seasonal Trends')
plt.ylabel('Mean Value')
plt.legend(title='Variables')
plt.show()

In [ ]:
for col in ['precip', 'pet', 'moisture_10cm', 'moisture_30cm', 'moisture_60cm', 'moisture_90cm', 'aet', 'gw_recharge']:
    sns.boxplot(x='season', y=col, data=df_seasonal)
    plt.title(f'Seasonal Distribution of {col}')
    plt.ylabel(col)
    plt.show()

# Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

# Load data and ensure the date column is in datetime format
df = pd.read_excel('./data/data.xlsx')
df['date'] = pd.to_datetime(df['date'])

# --- Create additional time-based features ---
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_year'] = df['date'].dt.dayofyear

# # Cyclical features for day_of_year to capture seasonality
# df['sin_day_of_year'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
# df['cos_day_of_year'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)

# --- Create hydrological features ---
# Difference between precipitation+irrigation and potential evapotranspiration (water balance)
df['precip_pet_diff'] = df['precipitation + irrigation (mm)'] - df['potential evapotranspiration (mm)']

# --- Create rolling window features ---
# 7-day rolling average and sum for precipitation and potential evapotranspiration
df['precip_7d_avg'] = df['precipitation + irrigation (mm)'].rolling(window=7, min_periods=1).mean()
df['pet_7d_avg'] = df['potential evapotranspiration (mm)'].rolling(window=7, min_periods=1).mean()

df['precip_7d_sum'] = df['precipitation + irrigation (mm)'].rolling(window=7, min_periods=1).sum()
df['pet_7d_sum'] = df['potential evapotranspiration (mm)'].rolling(window=7, min_periods=1).sum()

# --- Define inputs and outputs ---
input_features = [
    'date',
    'precipitation + irrigation (mm)',
    'potential evapotranspiration (mm)',
    'year',
    'month',
    'day',
    'day_of_week',
    'day_of_year',
    # 'sin_day_of_year',
    # 'cos_day_of_year',
    'precip_pet_diff',
    'precip_7d_avg',
    'pet_7d_avg',
    'precip_7d_sum',
    'pet_7d_sum'
]

output_features = [
    'depth 10cm',
    'depth 30cm',
    'depth 60cm',
    'depth 90cm',
    'actual evapotranspiration (mm)',
    'groundwater recharge (mm)'
]

# Create a new DataFrame for modeling
df_model = df[input_features + output_features].copy()

df_model

In [ ]:
# Save the enriched DataFrame to a CSV file
df_model.to_csv('./data/model_data.csv', index=False)